<h1>Hosting and Querying an Encrypted Model with Grid</h1>

## Import Dependencies

In [2]:
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

## Setup configs

In [7]:
# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
my_grid =  gr.GridNetwork("http://localhost:5000")

## Build model

In [5]:
# Build Model
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__(id="convnet")
        self.fc1 = th.nn.Linear(2, 1)
        self.bias = th.tensor([1000.0])
        self.state += ["fc1", "bias"]

    def forward(self, x):
        x = self.fc1(x)
        return x

model = Net()
model.build(th.tensor([1.0, 2]))

## Host the encrypted model on Grid Network

In [8]:
my_grid.serve_encrypted_model(model)

## Perform encrypted queries

In [10]:
x = th.tensor([1.0, 2])
result = my_grid.run_encrypted_inference('convnet', x)
print(result)

tensor([1.2290])
